# Imports

In [1]:
import numpy as np
import unittest
import math


from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from IPython.display import display, HTML
init_notebook_mode(connected=True)

# Define Components

## Vector Class

In [2]:
class Vector2d:
    def __init__(self, coords=None):
        ''' coords is array of size two
        if argument coords is not specified, then Vector2d will have random coordinates
        '''
        if type(coords) is Vector2d:
            raise ValueError('coords parameter is already a Vector2d object')
        
        if coords is None:
            self._coords = np.random.random(2)
        else:
            self._coords = np.array(coords)
            
    def get_x(self):
        return self._coords[0]
    
    def set_x(self, value):
        self._coords[0] = value

    def get_y(self):
        return self._coords[1]
    
    def set_y(self, value):
        self._coords[1] = value
    
    x = property()
    y = property()

    x = x.getter(get_x)
    x = x.setter(set_x)

    y = y.getter(get_y)
    y = y.setter(set_y)

    def __str__(self):
        return '[{0:,.3f}, {1:,.3f}] Vector2d'.format(self.x, self.y)
    
    def __repr__(self):
        return '[{0}, {1}] Vector2d'.format(self.x, self.y)
    
    def __eq__(self, other):
        return all(self._coords == other._coords)
        
    def __add__(self, other):
        return Vector2d(self._coords + other._coords)
    
    def __sub__(self, other):
        return Vector2d(self._coords - other._coords)
    
    @classmethod
    def dot(cls, v1, v2):
        return np.vdot(v1._coords, v2._coords)
    
    @classmethod
    def distance(cls, v1, v2):
        return math.sqrt(Vector2d.dot(v2 - v1, v2 - v1))
    
    def __rmul__(self, scalar):
        return Vector2d(scalar * self._coords)
    

In [3]:
v = Vector2d([4,6])
v.x = 10
v

[10, 6] Vector2d

## Environment Class

In [4]:
class Environment:
    def __init__(self, x, y, boundaries='no'):
        ''' 
        x and y are the size of our rectangular region.
        The center of the region is located at the (0, 0).
        The corners of the region are: (x/2, y/2), (x/2, -y/2), (-x/2, -y/2), (-x/2, y/2)
        (from top right, clockwise)
        '''
        self.width = x
        self.hight = y
        
    def wrap(self):
        pass
    

## Atom Class

In [5]:
class Atom:
    count = 0
    def __init__(self, mass, r=None, v=None, a=None):
        ''' Initialize r, v and a with random vectors if nothing is specified'''
        self.mass = mass
        self.n = Atom.count
        if r is None:
            self.r = Vector2d()
        else:
            self.r = Vector2d(r)
        
        if v is None:
            self.v = Vector2d()
        else:
            self.v = Vector2d(v)
            
        if a is None:
            self.a = Vector2d()
        else:
            self.a = Vector2d(a)
                
        Atom.count += 1

## Moleculalr Dynamics Class

In [32]:
class MolecularDynamics:
    def __init__(self, environment, mode='kinematics', dt=1, mols=None):
        self.env = environment
        if mols == None:
            self.mols = [
                Atom(0.1, [0.5,0], [1,1], [0,0]),
                Atom(0.1, [0,0.3], [-1,1], [0,0]),
                Atom(0.1, [1,1], [-1,-1], [0,0]),
            ]
        else:
            self.mols = mols
        self.dt = dt
        self.mode = mode.split('+')
        
    def leapfrog(self, part):
        if part == 1:
            for m in self.mols:
                m.v = m.v + 0.5 * self.dt * m.a
                m.r = m.r + self.dt * m.v
        elif part == 2:
            for m in self.mols:
                m.v = m.v + 0.5 * self.dt * m.a
    
    def leapfrogs(self):
        print('computing...')
        ''' the function makes one step and outputs molecules'''
        # new, not tested
        if 'kinematics' in self.mode: # lepfrgos only
            print('computing kinematics')
            self.leapfrog(1)
            self.leapfrog(2) 
        elif 'molecular' in self.mode and 'dynamics' in self.mode:
            print('computing molecular dynamics, but why ', self.mode) # that was a huge buggg, wasted so much time
            print("'molecular' and 'dynamics' in self.mode", 'molecular' and 'dynamics' in self.mode)
        elif 'dynamics' in self.mode: # acceleration and leapfrogs
            print('computing dynamics')
            self.compute_acceleration() 
            self.leapfrog(1)
            self.leapfrog(2)
        else:
            raise ValueError('Impossible to recognise mode')
        
        
        return self.mols
    
    def plot(self):
        mols_x = [m.r.x for m in self.mols]
        mols_y = [m.r.y for m in self.mols]
        
        trace = go.Scatter(
            x = mols_x,
            y = mols_y,
            mode = 'markers+text',
            text = list(map(str, list(range(1,len(self.mols)+1)))),
            textposition='bottom right',
            textfont=dict(
                family='sans serif',
                size=8,
                color='#ff7f0e'
            )
        )
        
        data = [trace]
        iplot(data)
    
    # not tested, under development
    def compute_acceleration(self):
        ''' computes forces for all the molecules
        '''
        for m in self.mols:
            m.a = Vector2d([0, 0])
        
        if 'gravity' in self.mode:
            print('\tcomputing gravity')
            for m in self.mols:
                m.a += Vector2d([0, -9.8]) # acceleration caused by gravity force
            
        if 'wind' in self.mode:
            print('\tcomputing wind')
            for m in self.mols:
                F = Vector2d([-0.1, 0]) # wind blows to the left
                m.a += 1/m.mass * F
    
    def apply_boundaries(self):
        ''' appies appropriate boundaries for molecules
        '''
        pass

# Launching

In [26]:
def build_coords(mols):
    ''' function that generates molecule coordinates for plots
    '''
    dictionary = {
        'x': [m.r.x for m in mols],
        'y': [m.r.y for m in mols]
    }
    return dictionary

r = lambda : (np.random.random() - 1) * 10
launch = True


if launch == True:
    env = Environment(1000, 1000)
    mols = [
        Atom(0.1, [0.5,0], [9,9], [r(),r()]),
        Atom(0.1, [0,0.3], [-19,1], [r(),r()]),
        Atom(0.1, [1,1], [-1,-19], [r(),r()]),
        Atom(0.1, [0.5,0.5], [19.5,1], [r(),r()]),
        Atom(0.1, [1,0.3], [-1,19], [r(),r()]),
        Atom(0.1, [3,1], [-19,-1], [r(),r()]),
    ]
    md = MolecularDynamics(env, dt=1, mols=mols, mode='kinematics')
    # [(md.plot(), md.leapfrog(1), md.leapfrog(2)) for i in range(12)]

    mols_x = [m.r.x for m in md.mols]
    mols_y = [m.r.y for m in md.mols]

    data=[dict(x=mols_x, y=mols_y, 
               mode='markers', 
               line=dict(width=2, color='blue')
              ),
        ]

    layout=dict(xaxis=dict(range=[-env.width, env.width], autorange=False, zeroline=False),
                yaxis=dict(range=[-env.hight, env.hight], autorange=False, zeroline=False, scaleanchor="x", scaleratio=1),
                title='Kinematic Generation',
                hovermode='closest',
               )

    
    frames=[dict(
        data=[
            {
                **(lambda mols: {'x': [m.r.x for m in mols], 'y': [m.r.y for m in mols]})(md.leapfrogs()),
                'mode': 'markers',
                'marker': dict(color='red', size=10)
            }]) for k in range(100)] 
    

    figure=dict(data=data, layout=layout, frames=frames)          
    iplot(figure, 'basic')

Atom.count

computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
c

8

# Unit Testing

## Vector Class

In [33]:
class VectorTest(unittest.TestCase):
    def setUp(self):
        self.arr = [Vector2d(), Vector2d(), Vector2d()]
        self.v2 = Vector2d([3,4.4])
        
    def test_coords(self):
        self.assertEqual(np.testing.assert_array_equal(np.array([3,4.4]), self.v2._coords), None)
        
    def test_random_vectors(self):
        print('Make sure that the vectors are different:\n',[v._coords for v in self.arr])
        self.assertEqual(9,9)


class VectorGetSet(unittest.TestCase):
    def setUp(self):
        self.v = Vector2d([10.2, 4.154])
    
    def test_set(self):
        self.v.x = 3
        self.v.y = 5
        self.assertEqual(np.testing.assert_array_equal(np.array([3,5]), self.v._coords), None)

    def test_get(self):
        self.assertEqual(self.v.x, 10.2)
        self.assertEqual(self.v.y, 4.154)



class VectorTestStrRepr(unittest.TestCase):
    def setUp(self):
        self.v = Vector2d([22.351355, -44.343143])
        
    def test_str(self):
        self.assertEqual(str(self.v), '[22.351, -44.343] Vector2d')
        
    def test_repr(self):
        self.assertEqual((self.v.__repr__()), '[22.351355, -44.343143] Vector2d')


class VectorEquality(unittest.TestCase):
    def setUp(self):
        self.v1 = Vector2d([22.351355, -44.343143])
        self.v2 = Vector2d([22.351355, -44.343143])
        self.v3 = Vector2d([33.3, -22.343143])
        
    def test_eq(self):
        '''testing __eq__ method
        '''
        self.assertEqual(self.v1, self.v2)
        
    def test_neq(self):
        self.assertNotEqual(self.v1, self.v3)


class VectorAddition(unittest.TestCase):
    def setUp(self):
        self.v1 = Vector2d([22.3, -44])
        self.v2 = Vector2d([2,4])
        
    def test_add(self):
        result = Vector2d([24.3, -40])
        self.assertEqual(self.v1 + self.v2, result)


class VectorSubtraction(unittest.TestCase):
    def setUp(self):
        self.v1 = Vector2d([22.3, -44])
        self.v2 = Vector2d([2,4])
        
    def test_add(self):
        result = Vector2d([20.3, -48])
        self.assertEqual(self.v1 - self.v2, result)
        

class VectorDot(unittest.TestCase):
    def setUp(self):
        self.v1 = Vector2d([11, -12])
        self.v2 = Vector2d([2,4])
        
    def test_dot(self):
        result = 11*2 - 12*4
        self.assertEqual(Vector2d.dot(self.v1, self.v2), result)    


class VectorDistance(unittest.TestCase):
    def setUp(self):
        self.v1 = Vector2d([11,-7])
        self.v2 = Vector2d([7,-4])

    def test_dist(self):
        result = 11*7 + 7*4
        self.assertEqual(Vector2d.dot(self.v1, self.v2), result)
        self.assertEqual(Vector2d.distance(self.v1, self.v2), 5)


class VectorRMul(unittest.TestCase):
    def setUp(self):
        self.v = Vector2d([11,-7])
        self.scalar = -2
        
    def test_rmul(self):
        self.assertEqual(self.scalar * self.v, Vector2d([-22,14]))

        
        
unittest.main(argv=['first-arg-is-ignored'], exit=False);

....

computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
c

.

['dynamics', 'wind', 'gravity']
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamic

..............

Make sure that the vectors are different:
 [array([0.82877848, 0.02012569]), array([0.65759105, 0.27770428]), array([0.52517559, 0.75079667])]



----------------------------------------------------------------------
Ran 19 tests in 0.355s

OK


## Environment Class

In [9]:
class TestEnvironment(unittest.TestCase):
    def setUp(self):
        self.env = Environment(3,5)
        
    def test_region(self):
        self.assertEqual((self.env.width, self.env.hight), (3,5))

unittest.main(argv=['first-arg-is-ignored'], exit=False);

..............

Make sure that the vectors are different:
 [array([0.3790416 , 0.39325932]), array([0.02409479, 0.52138458]), array([0.85782318, 0.4433271 ])]



----------------------------------------------------------------------
Ran 14 tests in 0.017s

OK


## Atom Class

In [10]:
class TestAtom(unittest.TestCase):
    def setUp(self):
        Atom.count = 0
        self.a1 = Atom(0.1)
        self.a2 = Atom(0.1)
        
    def test_randomness(self):
        a1 = self.a1
        a2 = self.a2

        self.assertNotEqual(a1.r, a1.a)
        self.assertNotEqual(a1.r, a1.a)
        self.assertNotEqual(a1.r, a2.r)

    def test_count1(self):
        self.assertEqual(2, Atom.count)
        # print(Atom.count) # 2
        # each test case starts with setUp
        # previous tests don't have any influence
        Atom(0.3, [3,6]), Atom(0.3, [3,6]), Atom(0.3, [3,6])
        self.assertEqual(5, Atom.count)

    def test_count2(self):
        self.assertEqual(2, Atom.count)
        # print(Atom.count)
        Atom(0.3, [3,6]), Atom(0.3, [3,6]), Atom(0.3, [3,6])
        self.assertEqual(5, Atom.count)
    
    def tearDown(self):
        Atom.count = 0

unittest.main(argv=['first-arg-is-ignored'], exit=False);

.................

Make sure that the vectors are different:
 [array([0.14044833, 0.67897201]), array([0.55630665, 0.09151343]), array([0.50707328, 0.72772686])]



----------------------------------------------------------------------
Ran 17 tests in 0.023s

OK


## Moleculalr Dynamics Class

In [31]:
class TestMD(unittest.TestCase):
    ''' Kinematics test
    '''
    def setUp(self):
        self.env = Environment(11, 11)
        self.r = np.array([-10, 10])
        self.v = np.array([7, 0])
        self.a = np.array([0, -9.8]) # let's say that we throw a stone
        self.atom = Atom(0.1, self.r, self.v, self.a)
        self.md = MolecularDynamics(env, mode='kinematics', dt=0.05, mols=[self.atom])
    
    def test_leapfrog(self):
        # real
        self.r_coords = []
        for i in range(100):
            self.r_coords.append(self.md.mols[0].r)
            self.md.leapfrogs()
        # print(self.r_coords)
        
        r_coords_x = [r.x for r in self.r_coords]
        r_coords_y = [r.y for r in self.r_coords]
        
        trace0 = go.Scatter(
            x=r_coords_x,
            y=r_coords_y,
            mode='markers',
            marker={'color': 'red', 'size': 11},
            name='Simulation',
        )
        
        # prediction
        time = np.linspace(0, 2, 200)

        r_coords = [self.r + t * self.v + self.a * t**2/2 for t in time]

        trace1 = go.Scatter(
            x=[r[0] for r in r_coords],
            y=[r[1] for r in r_coords], 
            marker={'color': 'blue'},
            name='Prediction',
        )
        
        data = [trace0, trace1]
        layout = {
            'xaxis': dict(range=[-11, -5], autorange=False, zeroline=False),
            'yaxis': dict(range=[-10, 11], autorange=False, zeroline=False, scaleanchor="x", scaleratio=1),
            'title': "$\\text{Simulation and prediction of }r(t)= r + v\cdot t + \\frac{a\cdot t^2}2$",
        }
        fig = {
            'data': data,
            'layout': layout,
        }
        
        iplot(fig, filename='falling')
        
        

unittest.main(argv=['first-arg-is-ignored'], exit=False); # don't forget to put it next time

....

computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
c

.E.............

['dynamics', 'wind', 'gravity']
computing...
computing dynamics
	computing gravity
Make sure that the vectors are different:
 [array([0.24537287, 0.12849433]), array([0.33434857, 0.4887574 ]), array([0.72871639, 0.77936749])]



ERROR: test_leapfrog_with_wind (__main__.TestMDDynamics)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-30-114be52c658a>", line 16, in test_leapfrog_with_wind
    self.md.leapfrogs()
  File "<ipython-input-28-1e065b0e765b>", line 37, in leapfrogs
    self.compute_acceleration()
  File "<ipython-input-28-1e065b0e765b>", line 76, in compute_acceleration
    m.a += Vector2d([0, -9.8]) # acceleration caused by gravity force
TypeError: unsupported operand type(s) for +=: 'int' and 'Vector2d'

----------------------------------------------------------------------
Ran 19 tests in 0.172s

FAILED (errors=1)


In [37]:
class TestMDDynamics(unittest.TestCase):
    def setUp(self):
        self.env = Environment(11, 11)
        self.r = np.array([-10, 10])
        self.v = np.array([7, 0])
        self.a = np.array([0, 0])
        self.atom = Atom(0.1, self.r, self.v, self.a)
        self.md = MolecularDynamics(env, mode='dynamics+wind+gravity', dt=0.05, mols=[self.atom])
        print(self.md.mode)
    
    def test_leapfrog_with_wind(self):
        # real
        self.r_coords = []
        for i in range(100):
            self.r_coords.append(self.md.mols[0].r)
            self.md.leapfrogs()
        # print(self.r_coords)
        
        r_coords_x = [r.x for r in self.r_coords]
        r_coords_y = [r.y for r in self.r_coords]
        
        trace0 = go.Scatter(
            x=r_coords_x,
            y=r_coords_y,
            mode='markers',
            marker={'color': 'red', 'size': 11},
            name='Simulation',
        )
        
        # prediction
        time = np.linspace(0, 2, 200)

        r_coords = [self.r + t * self.v + np.array([0,-9.8]) * t**2/2 for t in time]

        trace1 = go.Scatter(
            x=[r[0] for r in r_coords],
            y=[r[1] for r in r_coords], 
            marker={'color': 'blue'},
            name='Prediction',
        )
        
        data = [trace0, trace1]
        layout = {
            'xaxis': dict(range=[-11, -5], autorange=False, zeroline=False),
            'yaxis': dict(range=[-10, 11], autorange=False, zeroline=False, scaleanchor="x", scaleratio=1),
            'title': "$\\text{Simulation and prediction of }r(t)= r + v\cdot t + \\frac{a\cdot t^2}2$",
        }
        fig = {
            'data': data,
            'layout': layout,
        }
        
        iplot(fig, filename='falling')
        
        

unittest.main(argv=['first-arg-is-ignored'], exit=False); # don't forget to put it next time

....

computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
computing kinematics
computing...
c

.

['dynamics', 'wind', 'gravity']
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamics
	computing gravity
	computing wind
computing...
computing dynamic

..............

Make sure that the vectors are different:
 [array([0.3603155 , 0.21172796]), array([0.42859118, 0.30714694]), array([0.73681231, 0.47251819])]



----------------------------------------------------------------------
Ran 19 tests in 0.652s

OK
